# Running a Remote Rabi Experiment Series

The notebook below shows how to run a Rabi experiment, plotting the results into a Rabi frequency curve. The experiment requires a TopChef service to be active

Add this project's base directory to the path. Make sure that the TopChef client is available and importable

In [ ]:
import sys
import os

sys.path.append(os.path.abspath('../../..'))

Import the required modules to run the experiment

In [ ]:
from topchef_client import Client, Service, Job
from time import sleep
from datetime import datetime
import matplotlib.pyplot as plt
from collections.abc import Iterator
import numpy as np
from copy import deepcopy
import unittest
import unittest.mock as mock
from typing import Iterable, Dict, Union

Render matplotlib plots inline

In [ ]:
% matplotlib inline

## Experiment Runner

Define an experiment runner as sweeping between different pulse times in the pulse time parameter space

In [ ]:
class ExperimentRunner(Iterator):
    """
    Run several Rabi experiments, iterating over equally spaced intervals in the pulse
    time parameter space. This class accepts a minimum pulse time, a maximum pulse time,
    and the number of points to iterate over. Iterating over the runner will
    result in the experiments running
    """
    def __init__(
            self, 
            topchef_service: Service, 
            min_time: float=1e-14, 
            max_time: float=500e-9, 
            number_of_repetitions: int=50000, 
            number_of_points: int=100
    ) -> None:
        """
        Initialize the experiment
        
        :param TopChefService topchef_service: The service to use for running
            multiple experiments
        :param min_time: The minimum pulse time to use
        :param max_time: The maximum pulse time
        :param number_of_repetitions: The number of times that each experiment is to be repeated.
            The count number is the sum of photon counts received from the experiment
        :param number_of_points: The number of data points to use
        """
        self.min_time = min_time
        self.max_time = max_time
        self.number_of_points = number_of_points
        self.number_of_repetitions = number_of_repetitions
        
        self.last_iterated_index = 0
        
        self._topchef_service = topchef_service
        
    @property
    def parameter_space(self) -> Iterable:
        """
        
        :return: The array of points that should be iterated over
        """
        return np.linspace(self.min_time, self.max_time, self.number_of_points)
    
    @property
    def experiment_parameters(self) -> Dict[str, Union[str, int, float]]:
        """
        
        :return: The experiment parameters for the current experiment. The current
            experiment is determined by the last_iterated index
        """
        pulse_time = self.parameter_space[self.last_iterated_index]
        number_of_repetitions = self.number_of_repetitions
        
        return {
            'pulse_time': pulse_time,
            'number_of_repetitions': number_of_repetitions,
            'type': 'RABI'
        }
    
    def reset_iterator(self) -> None:
        """
        Prepare this object for iteration
        """
        self.last_iterated_index = 0
    
    @staticmethod
    def wait_until_job_done(job: Job) -> None:
        """
        Wait until the job is complete
        """
        while not job.is_complete:
            sleep(0.01)
    
    def __iter__(self) -> 'ExperimentRunner':
        """
        
        :return: An iterator that will iterate over the experiments in the parameter space
        """
        self.reset_iterator()
        return self
        
    def __len__(self) -> int:
        """
        
        :return: The number of data points in the experiment space
        """
        return len(self.parameter_space)
    
    def __next__(self) -> Dict[str, Union[str, int, float]]:
        """
        Increment the last_iterated_index, run the next experiment, wait for it
        to finish, and return the results
        
        :return: The results for one experiment
        """
        if self.last_iterated_index == len(self):
            raise StopIteration()
        else:
            print('Running experiment %s/%s' % (self.last_iterated_index + 1, len(self)))
            job = self._topchef_service.new_job(self.experiment_parameters)
            self.wait_until_job_done(job)
            self.last_iterated_index += 1
            return job.result
            

Define the experiment Parameters

## Run the thing we defined

In [ ]:
server_url = 'http://129.97.136.225:5000'
service_id = '1b98639a-6276-11e7-b3a3-0242ac110002'

In [ ]:
client = Client(server_url)
service = client.services[service_id]

In [ ]:
experiment_series = ExperimentRunner(service)

Burn baby burn!

In [ ]:
results = [result for result in experiment_series]

In [ ]:
for result in results:
    print(result)

## Plot the Results

In [ ]:
light_counts = [result["light_count"] for result in results]
dark_counts = [result["dark_count"] for result in results]
result_counts = [result["result_count"] for result in results]
parameter_space = experiment_series.parameter_space

In [ ]:
plt.figure(figsize=(15,6))
plt.plot(parameter_space, light_counts, label='light reference')
plt.plot(parameter_space, dark_counts, label='dark reference')
plt.plot(parameter_space, result_counts, label='experiment')
plt.legend()

plt.show()

### Test the class defined above

In [ ]:
class TestExperimentRunner(unittest.TestCase):
    """
    Contains unit tests for the experiment runner
    """
    def setUp(self):
        """
        Set up unit testing
        """
        self.service = mock.MagicMock()
        self.experiment = ExperimentRunner(self.service)
        self.experiment.wait_until_job_done = lambda x: x
    
    def test_parameter_mutation(self):
        """
        Test that changing the parameters changes the
        parameter space, and that changing the number of 
        points changes the length of the experiment series
        """
        self.experiment.min_time = 1e-11
        self.experiment.max_time = 500e-9
        self.experiment.number_of_points = 100
        
        self.assertEqual(len(self.experiment), 100)
        print('test passed')
        
    def test_parameter_space(self):
        """
        Visual test to check that changing the parameter space
        changes the experiment series that are displayed
        """
        self.experiment.min_time = 0
        self.experiment.max_time = 500e-9
        self.experiment.number_of_points = 100
        print(self.experiment.parameter_space)
        
        self.experiment.min_time = -100
        print(self.experiment.parameter_space)
        
    def test_parameter_display(self):
        """
        Tests that the for loop works correctly
        """
        results = [result for result in self.experiment]

case = TestExperimentRunner()
case.setUp()
case.test_parameter_mutation()
case.setUp()
case.test_parameter_display()

In [ ]:
case = TestExperimentRunner()
case.setUp()
case.test_parameter_space()